## DC 311 Requests Analysis
1. Analyze the # of 311 requests for each type before and after lock down (March 2020)
2. Exploratay analysis on FEMS requests

### Some findings:
1. 50 types of service requests changed significantly after lock down (p-val <0.05)
2. t-test results can be found at the result directory (../result/DC_service_type_ttest.csv) 
3. Box plots can be found at the figure directory (../figure/DC_ServiceTypeBoxPlt/)
4. The #311 requests that were assigned to FEMS decreased in 2020
5. There were no requests assigned to FEMS after April 2020 (maybe the procedure has been changed because of COVID-19)

In [1]:
import numpy as np
import pandas as pd
import os
import re
import glob
import csv
import warnings
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import math
warnings.filterwarnings("ignore")
DC_311_2019 = '/Users/julia/Box/Data_311/DMV_311/DMV_nonEM/DC/311_Washington_DC_2019.csv'
DC_311_2020 = '/Users/julia/Box/Data_311/DMV_311/DMV_nonEM/DC/311_Washington_DC_2020.csv'

### All 311 Requests in D.C during 2019 to 2020

1. DC 2019

In [2]:
DC_311_2019_df = pd.read_csv(DC_311_2019)
DC_311_2019_df['opened_date'] = pd.to_datetime(DC_311_2019_df['ADDDATE'], format='%Y/%m/%d %H:%M:%S')
DC_311_2019_df = DC_311_2019_df.loc[ DC_311_2019_df['opened_date'].dt.year == 2019]
DC_311_2019_df.drop_duplicates(subset=['OBJECTID'], inplace = True)
min_date = min(DC_311_2019_df['opened_date'])
max_date = max(DC_311_2019_df['opened_date'])
print("#311 reports between " + str(min_date) + " and " + str(max_date) + ": " + str(DC_311_2019_df.shape[0]))

#311 reports between 2019-01-01 05:15:41+00:00 and 2019-12-31 23:59:53+00:00: 364928


In [3]:
print(DC_311_2019_df.columns)

Index(['X', 'Y', 'OBJECTID', 'SERVICECODE', 'SERVICECODEDESCRIPTION',
       'SERVICETYPECODEDESCRIPTION', 'ORGANIZATIONACRONYM', 'SERVICECALLCOUNT',
       'ADDDATE', 'RESOLUTIONDATE', 'SERVICEDUEDATE', 'SERVICEORDERDATE',
       'INSPECTIONFLAG', 'INSPECTIONDATE', 'INSPECTORNAME',
       'SERVICEORDERSTATUS', 'STATUS_CODE', 'SERVICEREQUESTID', 'PRIORITY',
       'STREETADDRESS', 'XCOORD', 'YCOORD', 'LATITUDE', 'LONGITUDE', 'CITY',
       'STATE', 'ZIPCODE', 'MARADDRESSREPOSITORYID', 'WARD', 'DETAILS',
       'opened_date'],
      dtype='object')


In [4]:
DC_311_2020_df = pd.read_csv(DC_311_2020)
DC_311_2020_df['opened_date'] = pd.to_datetime(DC_311_2020_df['ADDDATE'], format='%Y/%m/%d %H:%M:%S')
DC_311_2020_df = DC_311_2020_df.loc[ DC_311_2020_df['opened_date'].dt.year == 2020]
DC_311_2020_df.drop_duplicates(subset=['OBJECTID'], inplace = True)
min_date = min(DC_311_2020_df['opened_date'])
max_date = max(DC_311_2020_df['opened_date'])
print("#311 reports between " + str(min_date) + " and " + str(max_date) + ": " + str(DC_311_2020_df.shape[0]))

#311 reports between 2020-01-01 05:00:38+00:00 and 2020-12-31 23:56:22+00:00: 304718


In [5]:
service_desc_2019 = DC_311_2019_df['SERVICECODEDESCRIPTION'].unique().tolist()
service_desc_2020 = DC_311_2020_df['SERVICECODEDESCRIPTION'].unique().tolist()

In [6]:
print(len(service_desc_2019))

133


In [7]:
print(len(service_desc_2020))

122


In [8]:
intersect_service_desc = set(service_desc_2019) & set(service_desc_2020)
print(len(intersect_service_desc))
intersect_service_desc

113


{'311Force Reported Issues',
 'Abandoned Bicycle',
 'Abandoned Vehicle - On Private Property',
 'Abandoned Vehicle - On Public Property',
 'Alley Cleaning',
 'Alley Repair',
 'Bicycle Services',
 'Bulk Collection',
 'Bus/Rail Issues',
 'Christmas Tree Collection Concerns',
 'Christmas Tree Removal-Seasonal',
 'Container Removal',
 'DC Facilities Landscaping and Grounds Maintenance(DGS)',
 'DC Government Information',
 'DCRA - Illegal Construction',
 'DCRA - Vacant Private Property Inspection',
 'DFHV - For Hire Vehicle Concerns',
 'DFHV - Lost and Found Claims (DC Taxi)',
 'DMV - Adjudication Supervisor',
 'DMV - Appeal',
 'DMV - Copy of Ticket',
 'DMV - Driver Record Issues',
 'DMV - Driver and Vehicle Services Refund',
 'DMV - Drivers License/ID Issues',
 'DMV - Drivers License/ID Reinstatement',
 'DMV - Forms, Applications, and Manuals Request',
 'DMV - Hearings',
 'DMV - Online Processing Issues',
 'DMV - Processing Center Manager',
 'DMV - Refunds - Tickets',
 'DMV - Ticket Paymen

#### Difference of service description between 2019 and 2020

In [9]:
sym_diff_service_desc = set(service_desc_2019) ^ set(service_desc_2020)
print(len(sym_diff_service_desc))
sym_diff_service_desc

29


{'Automation Activity',
 'Automation Activity Outcome',
 'Automation All Questions',
 'Automation Auto Close',
 'Automation Dep Picklist',
 'Automation Reallocate',
 'Automation Schedule',
 'Child Safety Seat Program',
 'Christmas Tree Removal-Missed',
 'Coronavirus (COVID-19) Mass Gathering Tracking',
 'DC Facilities Landscaping and Grounds Maintenance',
 'DDS - Improper Use of Restraints',
 'DGS-SRT-TEST',
 'DMV - Offset Tracking',
 'DOEE - Energy Benchmarking',
 'DOEE – Foam and Straw Ban',
 'FEMS - Honor Guard',
 'Illegal Fireworks',
 'Junk Motor Vehicle - CCS',
 'OUC NYE Test',
 'Quarantine Resource and Service Request (COVID-19)',
 'Rodent Inspection and Treatment- Follow Up',
 'Sample SR',
 'Shut the Door (DOEE)',
 'Snow Ticket Reimbursement',
 'Streetcar',
 'Test-Automation Activity',
 'Test-Provar-Service-Question',
 'Test-SW-Bulk Scheduled'}

In [10]:
diff_service_desc_2019 =  set(service_desc_2019) - set(service_desc_2020)
print(len(diff_service_desc_2019))
diff_service_desc_2019_ls = list(diff_service_desc_2019)

20


In [11]:
DC_311_2019_df.loc[DC_311_2019_df['SERVICECODEDESCRIPTION'].isin(diff_service_desc_2019_ls)].groupby('SERVICECODEDESCRIPTION').size()

SERVICECODEDESCRIPTION
Automation Activity                                   9
Automation Activity Outcome                           6
Automation All Questions                              1
Automation Auto Close                                 1
Automation Dep Picklist                               1
Automation Reallocate                                 2
Automation Schedule                                  10
Child Safety Seat Program                             8
DC Facilities Landscaping and Grounds Maintenance    15
DDS - Improper Use of Restraints                      2
DGS-SRT-TEST                                          2
DMV - Offset Tracking                                 9
DOEE - Energy Benchmarking                            1
DOEE – Foam and Straw Ban                            49
FEMS - Honor Guard                                    1
Junk Motor Vehicle - CCS                              4
OUC NYE Test                                          1
Rodent Inspection and Tre

In [12]:
diff_service_desc_2020 =  set(service_desc_2020) - set(service_desc_2019) 
print(len(diff_service_desc_2020))
diff_service_desc_2020_ls = list(diff_service_desc_2020)

9


In [13]:
DC_311_2020_df.loc[DC_311_2020_df['SERVICECODEDESCRIPTION'].isin(diff_service_desc_2020_ls)].groupby('SERVICECODEDESCRIPTION').size()

SERVICECODEDESCRIPTION
Christmas Tree Removal-Missed                          101
Coronavirus (COVID-19) Mass Gathering Tracking        1152
Illegal Fireworks                                      151
Quarantine Resource and Service Request (COVID-19)      17
Sample SR                                                9
Snow Ticket Reimbursement                                1
Test-Automation Activity                                 3
Test-Provar-Service-Question                             2
Test-SW-Bulk Scheduled                                  10
dtype: int64

In [14]:
DC_311_2019_df['SERVICETYPECODEDESCRIPTION'].unique()

array(['SWMA- Solid Waste Management Admistration',
       'Transportation Operations Administration',
       'PEMA- Parking Enforcement Management Administration',
       'Urban Forrestry', 'Department of Energy and Environment',
       'DOH- Department Of Health', 'Toa-Street & Bridge Maintenance',
       'Toa-Trans Sys Mnt-Signs', 'SIOD', 'SNOW', 'FEMS-Smoke Alarms',
       'Department of Transportation', 'Toa- Trans Sys Mnt',
       'Driver Vehicle Services', 'Adjudication Services',
       'Transportation Policy & Planning Administration',
       'HOMYDRPR- How Is My Driving Program', 'FEMS-Special Events',
       '311- Call Center', '311- Emergencies', nan,
       'Dept of Consumer & Regulatory Affairs',
       'Department of Disability Services', 'FEMS',
       'DFHV - Department of For-Hire Vehicles'], dtype=object)

In [15]:
DC_311_2020_df['SERVICETYPECODEDESCRIPTION'].unique()

array(['Transportation Operations Administration',
       'DOH- Department Of Health',
       'SWMA- Solid Waste Management Admistration',
       'PEMA- Parking Enforcement Management Administration',
       'Department of Energy and Environment', 'Toa-Trans Sys Mnt-Signs',
       'Toa-Street & Bridge Maintenance', 'Urban Forrestry',
       '311- Call Center', 'SIOD', 'Driver Vehicle Services',
       'Dept of Consumer & Regulatory Affairs', 'Toa- Trans Sys Mnt',
       'Adjudication Services', 'Department of Transportation',
       'Transportation Policy & Planning Administration', 'SNOW', nan,
       'HOMYDRPR- How Is My Driving Program', 'FEMS-Smoke Alarms', 'FEMS',
       'FEMS-Special Events', 'DFHV - Department of For-Hire Vehicles',
       '311- Emergencies', 'OUC - SNOW'], dtype=object)

In [16]:
DC_311_2019_df['added_month'] = DC_311_2019_df['opened_date'].dt.month
df = DC_311_2019_df.groupby(['added_month','SERVICECODEDESCRIPTION']).size().reset_index(name='count')
service_count_2019 = pd.DataFrame(columns = df['added_month'].unique().tolist(), index = df['SERVICECODEDESCRIPTION'].unique().tolist())
for row in range(df.shape[0]):
    setvice_type = df.at[row, 'SERVICECODEDESCRIPTION']
    month = df.at[row, 'added_month']
    service_count_2019.at[setvice_type, month] = df.at[row, 'count']
service_count_2019.fillna(0, inplace = True)
service_count_2019 = service_count_2019.add_prefix('2019_')
service_count_2019['total #reports_2019'] = service_count_2019.sum(axis=1)
service_count_2019.sort_values(by = 'total #reports_2019', ascending=False, inplace=True)
service_count_2019.reset_index(inplace=True)
service_count_2019.rename(columns = {'index': 'SERVICECODEDESCRIPTION'}, inplace = True)
service_count_2019.head(10)


,SERVICECODEDESCRIPTION,2019_1,2019_2,2019_3,2019_4,2019_5,2019_6,2019_7,2019_8,2019_9,2019_10,2019_11,2019_12,total #reports_2019
0,Bulk Collection,3979,3412,4601,5298,5563,5290,5437,4933,4464,4316,3816,4635,55744
1,Parking Enforcement,2825,2759,4167,4454,4974,4239,4152,4354,4092,4188,2950,3167,46321
2,Parking Meter Repair,3164,2883,3263,3452,3066,2269,1926,2670,2341,2079,1862,2750,31725
3,Emergency No-Parking Verification,1172,1277,1347,1807,1833,2018,2011,2120,1894,1785,1349,1158,19771
4,Pothole,2007,3267,3863,1153,887,628,506,436,234,283,303,545,14112
5,Trash Collection - Missed,2025,791,739,984,1113,1253,1798,1264,878,987,928,1098,13858
6,Roadway Signs,1055,1138,1050,1357,1198,1296,1453,1132,796,1044,856,826,13201
7,Illegal Dumping,1203,1210,882,832,947,1062,1254,1107,994,930,903,1012,12336
8,Residential Parking Permit Violation,812,710,1018,1117,1327,1022,1061,1222,1114,1123,898,855,12279
9,Sanitation Enforcement,834,832,738,803,1158,1112,1218,1055,908,725,600,634,10617


In [17]:
DC_311_2020_df['added_month'] = DC_311_2020_df['opened_date'].dt.month
df = DC_311_2020_df.groupby(['added_month','SERVICECODEDESCRIPTION']).size().reset_index(name='count')
service_count_2020 = pd.DataFrame(columns = df['added_month'].unique().tolist(), index = df['SERVICECODEDESCRIPTION'].unique().tolist())
for row in range(df.shape[0]):
    setvice_type = df.at[row, 'SERVICECODEDESCRIPTION']
    month = df.at[row, 'added_month']
    service_count_2020.at[setvice_type, month] = df.at[row, 'count']
service_count_2020.fillna(0, inplace = True)
service_count_2020 = service_count_2020.add_prefix('2020_')
service_count_2020['total #reports_2020'] = service_count_2020.sum(axis=1)
service_count_2020.sort_values(by = 'total #reports_2020', ascending=False, inplace=True)
service_count_2020.reset_index(inplace=True)
service_count_2020.rename(columns = {'index': 'SERVICECODEDESCRIPTION'}, inplace = True)
service_count_2020.head(10)

,SERVICECODEDESCRIPTION,2020_1,2020_2,2020_3,2020_4,2020_5,2020_6,2020_7,2020_8,2020_9,2020_10,2020_11,2020_12,total #reports_2020
0,Bulk Collection,4533,3495,4518,4868,6409,6463,6759,6613,6442,5665,5301,5269,66335
1,Parking Enforcement,3063,3015,2485,1291,1295,1511,1559,1772,1749,2118,1493,1590,22941
2,Trash Collection - Missed,1444,781,977,1219,1131,1328,1597,1842,1458,1392,1434,1254,15857
3,Illegal Dumping,1165,941,748,727,1024,1146,1293,1365,1394,1112,810,1083,12808
4,DC Government Information,151,65,934,1276,1080,1313,1333,1201,995,1294,896,1008,11546
5,Roadway Signs,1124,1421,1112,1508,867,618,841,763,970,745,698,659,11326
6,Streetlight Repair Investigation,878,797,555,605,681,744,865,861,921,887,1027,968,9789
7,Rodent Inspection and Treatment,502,507,467,659,938,990,917,863,910,846,638,515,8752
8,Parking Meter Repair,2174,2096,1215,136,198,313,336,360,440,410,311,176,8165
9,Sanitation Enforcement,885,721,636,589,742,675,719,773,780,547,403,492,7962


In [18]:
# remove the last column
merge_service_count = pd.merge(service_count_2019.iloc[:,0:-1], service_count_2020.iloc[:,0:-1], on = 'SERVICECODEDESCRIPTION', how = 'outer')
merge_service_count.fillna(0, inplace = True)
merge_service_count.head()

,SERVICECODEDESCRIPTION,2019_1,2019_2,2019_3,2019_4,2019_5,2019_6,2019_7,2019_8,2019_9,...,2020_3,2020_4,2020_5,2020_6,2020_7,2020_8,2020_9,2020_10,2020_11,2020_12
0,Bulk Collection,3979.0,3412.0,4601.0,5298.0,5563.0,5290.0,5437.0,4933.0,4464.0,...,4518.0,4868.0,6409.0,6463.0,6759.0,6613.0,6442.0,5665.0,5301.0,5269.0
1,Parking Enforcement,2825.0,2759.0,4167.0,4454.0,4974.0,4239.0,4152.0,4354.0,4092.0,...,2485.0,1291.0,1295.0,1511.0,1559.0,1772.0,1749.0,2118.0,1493.0,1590.0
2,Parking Meter Repair,3164.0,2883.0,3263.0,3452.0,3066.0,2269.0,1926.0,2670.0,2341.0,...,1215.0,136.0,198.0,313.0,336.0,360.0,440.0,410.0,311.0,176.0
3,Emergency No-Parking Verification,1172.0,1277.0,1347.0,1807.0,1833.0,2018.0,2011.0,2120.0,1894.0,...,1651.0,133.0,4.0,3.0,67.0,87.0,59.0,3.0,4.0,11.0
4,Pothole,2007.0,3267.0,3863.0,1153.0,887.0,628.0,506.0,436.0,234.0,...,490.0,283.0,335.0,245.0,214.0,285.0,199.0,158.0,150.0,227.0


In [19]:
merge_service_count.columns

Index(['SERVICECODEDESCRIPTION', '2019_1', '2019_2', '2019_3', '2019_4',
       '2019_5', '2019_6', '2019_7', '2019_8', '2019_9', '2019_10', '2019_11',
       '2019_12', '2020_1', '2020_2', '2020_3', '2020_4', '2020_5', '2020_6',
       '2020_7', '2020_8', '2020_9', '2020_10', '2020_11', '2020_12'],
      dtype='object')

In [20]:
pre_covid_cols = ['SERVICECODEDESCRIPTION', '2019_1', '2019_2', '2019_3', '2019_4','2019_5', '2019_6', '2019_7', '2019_8', '2019_9', '2019_10', '2019_11','2019_12', '2020_1', '2020_2']
covid_cols = ['SERVICECODEDESCRIPTION', '2020_3', '2020_4', '2020_5', '2020_6','2020_7', '2020_8', '2020_9', '2020_10', '2020_11','2020_12']
pre_covid_service_count = merge_service_count[pre_covid_cols]
covid_service_count = merge_service_count[covid_cols]

In [21]:
pre_covid_service_count['total #reports before lock down'] = pre_covid_service_count.sum(axis=1)
covid_service_count['total #reports after lock down'] = covid_service_count.sum(axis=1)

In [22]:
pre_covid_service_count.sort_values(by = 'total #reports before lock down', ascending=False, inplace=True)
pre_covid_service_count.reset_index(drop= True, inplace=True)

In [23]:
covid_service_count.sort_values(by = 'total #reports after lock down', ascending=False, inplace=True)
covid_service_count.reset_index(drop= True, inplace=True)

In [24]:
pre_covid_types = pre_covid_service_count['SERVICECODEDESCRIPTION'].tolist()
covid_types = covid_service_count['SERVICECODEDESCRIPTION'].tolist()
pre_covid_service_count

,SERVICECODEDESCRIPTION,2019_1,2019_2,2019_3,2019_4,2019_5,2019_6,2019_7,2019_8,2019_9,2019_10,2019_11,2019_12,2020_1,2020_2,total #reports before lock down
0,Bulk Collection,3979.0,3412.0,4601.0,5298.0,5563.0,5290.0,5437.0,4933.0,4464.0,4316.0,3816.0,4635.0,4533.0,3495.0,63772.0
1,Parking Enforcement,2825.0,2759.0,4167.0,4454.0,4974.0,4239.0,4152.0,4354.0,4092.0,4188.0,2950.0,3167.0,3063.0,3015.0,52399.0
2,Parking Meter Repair,3164.0,2883.0,3263.0,3452.0,3066.0,2269.0,1926.0,2670.0,2341.0,2079.0,1862.0,2750.0,2174.0,2096.0,35995.0
3,Emergency No-Parking Verification,1172.0,1277.0,1347.0,1807.0,1833.0,2018.0,2011.0,2120.0,1894.0,1785.0,1349.0,1158.0,1165.0,1558.0,22494.0
4,Trash Collection - Missed,2025.0,791.0,739.0,984.0,1113.0,1253.0,1798.0,1264.0,878.0,987.0,928.0,1098.0,1444.0,781.0,16083.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,Test-SW-Bulk Scheduled,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138,Sample SR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
139,Test-Automation Activity,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
140,Test-Provar-Service-Question,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
total_service_types = list(set(pre_covid_types + covid_types))
len(total_service_types)

142

### Perform t-test on #reports for each type across months
* To understand the difference before and after lock down (lock down: March - Dec. 2020)

In [26]:
t_test_df = pd.DataFrame(columns=['service_type','p-value','direction'])
for service_type in total_service_types:
    pre_covid_var = pre_covid_service_count.loc[pre_covid_service_count['SERVICECODEDESCRIPTION']== service_type][pre_covid_cols[1:]].values.flatten()
    covid_var = covid_service_count.loc[covid_service_count['SERVICECODEDESCRIPTION']== service_type][covid_cols[1:]].values.flatten()
    t_val, p_val = stats.ttest_ind(pre_covid_var, covid_var, equal_var = False)
    direction = covid_var.mean() - pre_covid_var.mean()
    t_test_res = {'service_type': service_type, 'p-value': p_val, 'direction': direction}
    t_test_df = t_test_df.append(t_test_res,ignore_index = True)

In [27]:
t_test_df.sort_values(by = 'p-value', inplace=True)
t_test_df.reset_index(drop=True, inplace = True)
t_test_df

,service_type,p-value,direction
0,Parking Meter Repair,2.217613e-11,-2181.571429
1,Residential Parking Permit Violation,5.841563e-09,-882.785714
2,Parking Enforcement,1.776576e-08,-2056.485714
3,FEMS - Community Events,6.265158e-07,-58.871429
4,Emergency No-Parking Verification,1.880452e-06,-1404.514286
...,...,...,...
137,DMV - Vehicle Title Issues,8.658522e-01,-0.542857
138,DFHV - For Hire Vehicle Concerns,8.845421e-01,0.057143
139,311Force Reported Issues,9.496550e-01,0.014286
140,Grass and Weeds Mowing,9.524280e-01,-2.500000


In [28]:
t_test_df.to_csv('../result/DC_service_type_ttest.csv')

In [29]:
t_test_low_p = t_test_df.loc[t_test_df['p-value']<=0.05]
t_test_low_p.sort_values(by = ['direction','p-value'], inplace=True)
t_test_low_p.reset_index(drop = True, inplace = True)
t_test_low_p['inc/dec'] =['increased' if sign == True else 'decreased' for sign in (t_test_low_p['direction'] >0).tolist()]
inc_dec_df = t_test_low_p.groupby('inc/dec').size().reset_index()
inc_dec_df.rename(columns = {0:'#service types'}, inplace = True)
inc_dec_df

,inc/dec,#service types
0,decreased,27
1,increased,23


In [30]:
t_test_low_p.head(10)

,service_type,p-value,direction,inc/dec
0,Parking Meter Repair,2.217613e-11,-2181.571429,decreased
1,Parking Enforcement,1.776576e-08,-2056.485714,decreased
2,Emergency No-Parking Verification,1.880452e-06,-1404.514286,decreased
3,Residential Parking Permit Violation,5.841563e-09,-882.785714,decreased
4,Pothole,1.635736e-02,-843.400000,decreased
5,Roadway Signs,2.657126e-02,-246.614286,decreased
6,Sanitation Enforcement,1.557205e-03,-237.471429,decreased
7,Out of State Parking Violation (ROSA),1.122262e-04,-199.871429,decreased
8,Roadway Repair,1.447568e-03,-94.028571,decreased
9,FEMS - Community Events,6.265158e-07,-58.871429,decreased


### Box plot for types that have p-value <0.05

In [31]:
low_p_types = t_test_low_p['service_type'].tolist()

In [32]:
pre_covid_box_plt = pre_covid_service_count.set_index('SERVICECODEDESCRIPTION').T.iloc[:-1].reset_index()

covid_box_plt = covid_service_count.set_index('SERVICECODEDESCRIPTION').T.iloc[:-1].reset_index()
covid_box_plt = covid_box_plt[pre_covid_box_plt.columns.to_list()]
box_plt_df = pre_covid_box_plt.append(covid_box_plt,ignore_index=True,sort=False)
box_plt_df.rename(columns={'index':'year_month'},inplace = True)
period_ls = ['before lock down']* (pre_covid_box_plt.shape[0])
period_ls.extend(['after lock down']* (covid_box_plt.shape[0]))
box_plt_df['period'] = period_ls
box_plt_df.head(10)

SERVICECODEDESCRIPTION,year_month,Bulk Collection,Parking Enforcement,Parking Meter Repair,Emergency No-Parking Verification,Trash Collection - Missed,Roadway Signs,Pothole,Residential Parking Permit Violation,Illegal Dumping,...,Automation Dep Picklist,Coronavirus (COVID-19) Mass Gathering Tracking,Illegal Fireworks,Quarantine Resource and Service Request (COVID-19),Test-SW-Bulk Scheduled,Sample SR,Test-Automation Activity,Test-Provar-Service-Question,Snow Ticket Reimbursement,period
0,2019_1,3979.0,2825.0,3164.0,1172.0,2025.0,1055.0,2007.0,812.0,1203.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,before lock down
1,2019_2,3412.0,2759.0,2883.0,1277.0,791.0,1138.0,3267.0,710.0,1210.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,before lock down
2,2019_3,4601.0,4167.0,3263.0,1347.0,739.0,1050.0,3863.0,1018.0,882.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,before lock down
3,2019_4,5298.0,4454.0,3452.0,1807.0,984.0,1357.0,1153.0,1117.0,832.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,before lock down
4,2019_5,5563.0,4974.0,3066.0,1833.0,1113.0,1198.0,887.0,1327.0,947.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,before lock down
5,2019_6,5290.0,4239.0,2269.0,2018.0,1253.0,1296.0,628.0,1022.0,1062.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,before lock down
6,2019_7,5437.0,4152.0,1926.0,2011.0,1798.0,1453.0,506.0,1061.0,1254.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,before lock down
7,2019_8,4933.0,4354.0,2670.0,2120.0,1264.0,1132.0,436.0,1222.0,1107.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,before lock down
8,2019_9,4464.0,4092.0,2341.0,1894.0,878.0,796.0,234.0,1114.0,994.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,before lock down
9,2019_10,4316.0,4188.0,2079.0,1785.0,987.0,1044.0,283.0,1123.0,930.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,before lock down


In [33]:
num_imgs = math.ceil(len(low_p_types)/4)
sns.set_theme(style="whitegrid")
type_count =0
for img in range(num_imgs):
    if img < num_imgs -1:
        rows = 2
    else:
        rows = int(math.ceil(len(low_p_types) - type_count) /2)
    fig, axes = plt.subplots(rows,2, figsize = (20,10))
    for row in range(rows):
        for col in range(2):
            if rows >1:
                sns.boxplot(x='period', y=low_p_types[type_count], data=box_plt_df, ax=axes[row, col])
            else:
                sns.boxplot(x='period', y=low_p_types[type_count], data=box_plt_df, ax=axes[col])
            type_count +=1
    plt.savefig('../figure/DC_ServiceTypesBoxPlt/box_plt_' + str(img +1))
    plt.close()

## Analysis of 311 calls that were assigned to FEMS

In [34]:
DC_311_2019_df.ORGANIZATIONACRONYM.unique().tolist()

['DPW',
 'DDOT',
 'DOEE',
 'DOH',
 'FEMS',
 'DMV',
 'ORM',
 'OUC',
 'DCRA',
 nan,
 'DDS',
 'DGS',
 'DFHV',
 '311 Customer Service Center']

### 311 calls that were assigned to FEMS during 2019

In [35]:
DC_311_2019_fems = DC_311_2019_df.loc[DC_311_2019_df['ORGANIZATIONACRONYM'] == 'FEMS']
DC_311_2019_fems

,X,Y,OBJECTID,SERVICECODE,SERVICECODEDESCRIPTION,SERVICETYPECODEDESCRIPTION,ORGANIZATIONACRONYM,SERVICECALLCOUNT,ADDDATE,RESOLUTIONDATE,...,LATITUDE,LONGITUDE,CITY,STATE,ZIPCODE,MARADDRESSREPOSITORYID,WARD,DETAILS,opened_date,added_month
91,-76.925117,38.886951,279126,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2019/01/04 17:06:41+00,2019/01/07 15:06:27+00,...,38.886944,-76.925115,WASHINGTON,DC,20019.0,10428.0,7.0,NaN,2019-01-04 17:06:41+00:00,1
539,-76.960797,38.877512,279574,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2019/01/16 22:27:57+00,2019/03/23 16:02:28+00,...,38.877504,-76.960795,WASHINGTON,DC,20019.0,50666.0,7.0,FIREFIGHTERS NEVER SHOWED UP.,2019-01-16 22:27:57+00:00,1
707,-76.981361,38.922836,279742,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2019/01/18 19:50:06+00,2019/01/23 21:05:37+00,...,38.922828,-76.981359,WASHINGTON,DC,20018.0,61311.0,5.0,NaN,2019-01-18 19:50:06+00:00,1
1051,-77.020444,38.961513,280086,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2019/01/15 18:43:00+00,2019/02/01 19:23:58+00,...,38.961505,-77.020442,WASHINGTON,DC,20011.0,246463.0,4.0,REQUEST SMOKE ALARMS.,2019-01-15 18:43:00+00:00,1
1187,-76.993583,38.892218,280222,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2019/01/11 19:02:21+00,2019/03/14 17:30:29+00,...,38.892211,-76.993581,WASHINGTON,DC,20002.0,81651.0,6.0,"SMOKE ALARMS, INSTALLED.",2019-01-11 19:02:21+00:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359992,-77.022480,38.930865,716835,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2019/12/26 14:17:21+00,2020/01/17 20:15:20+00,...,38.930858,-77.022478,WASHINGTON,DC,20010.0,228942.0,1.0,NaN,2019-12-26 14:17:21+00:00,12
360159,-77.000487,38.962512,717002,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2019/12/26 21:31:56+00,2020/01/03 14:58:46+00,...,38.962505,-77.000485,WASHINGTON,DC,20011.0,25978.0,4.0,NaN,2019-12-26 21:31:56+00:00,12
363225,-77.019651,38.958943,720116,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2019/12/30 19:19:01+00,2020/01/27 19:52:12+00,...,38.958936,-77.019648,WASHINGTON,DC,20011.0,248819.0,4.0,NaN,2019-12-30 19:19:01+00:00,12
363470,-76.995047,38.853735,720361,FEMSFIREINSP,FEMS - Fire Safety Inspection,FEMS,FEMS,1,2019/12/30 21:40:15+00,2019/12/30 22:22:11+00,...,38.853728,-76.995045,WASHINGTON,DC,20032.0,301360.0,8.0,NaN,2019-12-30 21:40:15+00:00,12


### 311 calls that were assigned to FEMS during 2020

In [36]:
DC_311_2020_fems = DC_311_2020_df.loc[DC_311_2020_df['ORGANIZATIONACRONYM'] == 'FEMS']
DC_311_2020_fems

,X,Y,OBJECTID,SERVICECODE,SERVICECODEDESCRIPTION,SERVICETYPECODEDESCRIPTION,ORGANIZATIONACRONYM,SERVICECALLCOUNT,ADDDATE,RESOLUTIONDATE,...,LATITUDE,LONGITUDE,CITY,STATE,ZIPCODE,MARADDRESSREPOSITORYID,WARD,DETAILS,opened_date,added_month
4948,-77.000547,38.962513,6405,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2020/01/04 23:27:47+00,2020/01/16 20:52:53+00,...,38.962505,-77.000545,WASHINGTON,DC,20011.0,26070.0,4.0,NaN,2020-01-04 23:27:47+00:00,1
5495,-76.935298,38.902693,7208,FEMEDUPR,FEMS - Fire Safety Education,FEMS,FEMS,1,2020/01/06 12:29:45+00,2020/02/20 14:39:52+00,...,38.902686,-76.935296,WASHINGTON,DC,20019.0,18109.0,7.0,NaN,2020-01-06 12:29:45+00:00,1
5561,-76.988792,38.835090,7274,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2020/01/06 14:17:17+00,2020/01/27 19:53:13+00,...,38.835083,-76.988789,WASHINGTON,DC,20032.0,279187.0,8.0,NaN,2020-01-06 14:17:17+00:00,1
6401,-76.936911,38.880609,8306,FESPEDNU,FEMS - Community Events,FEMS-Special Events,FEMS,1,2020/01/06 18:58:23+00,2020/01/10 19:00:26+00,...,38.880602,-76.936908,WASHINGTON,DC,20019.0,21041.0,7.0,NaN,2020-01-06 18:58:23+00:00,1
6471,-76.991575,38.903698,8376,FESPEDNU,FEMS - Community Events,FEMS-Special Events,FEMS,1,2020/01/06 17:49:35+00,2020/01/09 15:13:11+00,...,38.903690,-76.991573,WASHINGTON,DC,20002.0,285821.0,5.0,NaN,2020-01-06 17:49:35+00:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79629,-76.993517,38.881809,111901,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2020/03/27 18:59:13+00,2020/04/09 13:54:48+00,...,38.881801,-76.993515,WASHINGTON,DC,20003.0,10968.0,6.0,NaN,2020-03-27 18:59:13+00:00,3
80583,-77.004331,38.914935,113639,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2020/03/29 03:02:57+00,2020/04/09 13:54:48+00,...,38.914927,-77.004328,WASHINGTON,DC,20002.0,296345.0,5.0,NaN,2020-03-29 03:02:57+00:00,3
84250,-77.021938,38.957644,118682,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2020/04/03 05:15:51+00,2020/04/09 13:54:48+00,...,38.957636,-77.021936,WASHINGTON,DC,20011.0,246643.0,4.0,NaN,2020-04-03 05:15:51+00:00,4
88092,-76.990267,38.950702,125308,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2020/04/08 18:51:25+00,2020/04/09 13:54:48+00,...,38.950694,-76.990264,WASHINGTON,DC,20017.0,15424.0,5.0,NaN,2020-04-08 18:51:25+00:00,4


In [37]:
print("#311 reports that were assigned to FEMS in 2019: " + str(DC_311_2019_fems.shape[0]))
print("#311 reports that were assigned to FEMS in 2020: " + str(DC_311_2020_fems.shape[0]))

print("FEMS reports/ total #311 in 2019 " + str(DC_311_2019_fems.shape[0] / DC_311_2019_df.shape[0]))
print("FEMS reports/ total #311 in 2020 " + str(DC_311_2020_fems.shape[0] / DC_311_2020_df.shape[0]))

#311 reports that were assigned to FEMS in 2019: 1254
#311 reports that were assigned to FEMS in 2020: 160
FEMS reports/ total #311 in 2019 0.003436294282707822
FEMS reports/ total #311 in 2020 0.0005250756437099219


In [38]:
DC_311_2019_fems['add_month'] = DC_311_2019_fems['opened_date'].dt.month
DC_311_2019_fems.groupby('add_month').size()

add_month
1     124
2      77
3     104
4     126
5     119
6     125
7     111
8     105
9     102
10    131
11     79
12     51
dtype: int64

In [39]:
DC_311_2020_fems['add_month'] = DC_311_2020_fems['opened_date'].dt.month
DC_311_2020_fems.groupby('add_month').size()

add_month
1    55
2    72
3    30
4     3
dtype: int64